This notebook aims at reproducing the training and results for the worker object detection. It also computes the performance of the model. More specifically, we use the detecto python library to train our model (https://detecto.readthedocs.io/en/latest/)

# Imports

In [ ]:
from utils.preprocessing import get_kwargs_xml_worker, create_xml
from utils.performance_viz import plot_matching_bbs, get_mean_iou, plot_bbs
from utils.performance_metrics import get_avg_precision_at_iou, plot_pr_curve, COLORS

import glob
import os
import random
from shutil import copyfile

from detecto import core, utils, visualize
from torchvision import transforms

import numpy as np

import xmltodict
import pickle
import matplotlib.pyplot as plt

import time

# Preprocessing

This part assumes that we already have downloaded the images and jsons related to this project. \
If not, please refer to the README to download these files. 

## Convert JSON to XML

In [ ]:
# Define directories from root (here /home)
imgs_directory = "/home/jovyan/chronsite/Detection_Train_Set/Detection_Train_Set_Img" 
jsons_directory = "/home/jovyan/chronsite/Detection_Train_Set/Detection_Train_Set_Json"
xml_directory = "/home/jovyan/chronsite/Detection_Train_Set/Detection_Train_Set_Xml"

imgs = glob.glob(imgs_directory + "/*.jpg")
jsons = glob.glob(jsons_directory + "/*.json")

In [ ]:
try:
    os.mkdir(xml_directory)
except FileExistsError:
    print("Directory already created")

In [ ]:
for json_path in jsons:
    kwgs = get_kwargs_xml_worker(json_path)
    create_xml(xml_directory, **kwgs)

## Train Test Split

To evaluate the performance of our model, we create a training (80%) and testing (20%) dataset. 

In [ ]:
# Create directories
os.mkdir("train")
os.mkdir("test")

# Training Dataset
tot_imgs = len(imgs)
random.seed(41)
train_imgs = random.sample(imgs, k=round(tot_imgs*0.8))
for img_path in train_imgs:
    img_name = img_path.split("/")[-1]
    xml_path = xml_directory + "/" + img_name.replace(".jpg", ".xml")
    copyfile(img_path, "train/" + img_name)
    copyfile(xml_path, "train/" + img_name.replace(".jpg", ".xml"))

# Testing dataset
test_imgs = [i for i in imgs if i not in train_imgs]
for img_path in test_imgs:
    img_name = img_path.split("/")[-1]
    xml_path = xml_directory + "/" + img_name.replace(".jpg", ".xml")
    copyfile(img_path, "test/" + img_name)
    copyfile(xml_path, "test/" + img_name.replace(".jpg", ".xml"))

# Training

In [ ]:
custom_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(800),
    transforms.ToTensor(),
    utils.normalize_transform()
])

dataset = core.Dataset('train/', transform=custom_transforms)
val_dataset = core.Dataset('test/', transform=custom_transforms)
loader = core.DataLoader(dataset, batch_size=8, shuffle=True)

model = core.Model(['worker']) # Label of what we try to detect (same as value in xml file)
losses = model.fit(loader, val_dataset, epochs=3, learning_rate=0.001, 
                   lr_step_size=5, verbose=True)

In [ ]:
# To avoid training again, we can load the model directly here:
model = core.Model.load('final_model.pth', ["worker"])

# Performance

In [ ]:
train_imgs = glob.glob("train/*.jpg")
test_imgs = glob.glob("test/*.jpg")

## Visually

In [ ]:
img_path = random.choice(train_imgs)

In [ ]:
plot_bbs(img_path, model, eps_proba=0.15)

The first plot shows all the predicted bounding boxes (with eps higher than smth), while the second plot below shows only the predicted bounding boxes (with eps higher than smth) that we were able to match to a true bounding box. 

In [ ]:
obj = plot_matching_bbs(img_path, model, eps_proba=0.1, eps_iou=0.1)

**Green** boxes are the true bounding boxes. \
**Blue** boxes are the predicted ones. \
***Note:*** Among the predicted boxes, we only consider the boxes having an IOU higher than eps_iou.

In [ ]:
print("The mean IOU for this image is", np.round(get_mean_iou(img_path, model, eps_proba=0.1, eps_iou=0), 2))

## Metrics

### Mean IOU

In [ ]:
ious_train = []
for img_path in train_imgs:
    iou = get_mean_iou(img_path, model, eps=0.1)
    ious_train.append(iou)
    print(iou)

In [ ]:
print("The mean IOU for training images is", np.nanmean(ious_train))

### Mean Average Precision (mAP)

In [ ]:
# Dictionary for true box

true_box = {}
for img_path in train_imgs:
    boxes = []
    img_name = img_path.split("/")[-1]
    
    xml_path = img_path.replace(".jpg", ".xml")
    with open(xml_path) as fd:
        doc = xmltodict.parse(fd.read())
        
    try:
        for i, obj in enumerate(doc["annotation"]["object"]):
            xmin = float(obj["bndbox"]["xmin"])
            xmax = float(obj["bndbox"]["xmax"])
            ymin = float(obj["bndbox"]["ymin"])
            ymax = float(obj["bndbox"]["ymax"])
            boxes.append([xmin, ymin, xmax, ymax])
            
    except KeyError: # When no object
        pass
    
    except TypeError: # When only 1 object
        xmin = float(doc["annotation"]["object"]["bndbox"]["xmin"])
        xmax = float(doc["annotation"]["object"]["bndbox"]["xmax"])
        ymin = float(doc["annotation"]["object"]["bndbox"]["ymin"])
        ymax = float(doc["annotation"]["object"]["bndbox"]["ymax"])
        boxes.append([xmin, ymin, xmax, ymax])
    
    true_box[img_name] = boxes

In [ ]:
# Dictionary for pred box

pred_box = {}
for img_path in train_imgs:
    img_name = img_path.split("/")[-1]
    pred_box[img_name] = {}
    
    image = utils.read_image(img_path)
    labels, bbs, scores = model.predict(image)

    boxes_list = []
    scores_list = []
    for tensor, score in zip(bbs, scores):
        xmin, ymin, xmax, ymax = tensor.numpy()
        boxes_list.append([xmin, ymin, xmax, ymax])
        scores_list.append(score.numpy().item())
    
    pred_box[img_name]["boxes"] = boxes_list
    pred_box[img_name]["scores"] = scores_list

In [ ]:
# pickle.dump(pred_box, open("dict_train.pkl", "wb"))
pred_box = pickle.load(open("dict_train.pkl", "rb"))

In [ ]:
# Precision Recall Curve

iou_thr = 0.7
start_time = time.time()
data = get_avg_precision_at_iou(true_box, pred_box, iou_thr=iou_thr)
end_time = time.time()
print('Single IoU calculation took {:.4f} secs'.format(end_time - start_time))
print('avg precision: {:.4f}'.format(data['avg_prec']))

start_time = time.time()
ax = None
avg_precs = []
iou_thrs = []
for idx, iou_thr in enumerate(np.linspace(0.1, 0.9, 9)):
    data = get_avg_precision_at_iou(true_box, pred_box, iou_thr=iou_thr)
    avg_precs.append(data['avg_prec'])
    iou_thrs.append(iou_thr)

    precisions = data['precisions']
    recalls = data['recalls']
    ax = plot_pr_curve(
        precisions, recalls, label='{:.2f}'.format(iou_thr), color=COLORS[idx], ax=ax)

# Prettify
avg_precs = [float('{:.4f}'.format(ap)) for ap in avg_precs]
iou_thrs = [float('{:.4f}'.format(thr)) for thr in iou_thrs]
print('map: {:.2f}'.format(100*np.mean(avg_precs)))
print('avg precs: ', avg_precs)
print('iou_thrs:  ', iou_thrs)
plt.legend(loc='upper right', title='IOU Thr', frameon=True)
for xval in np.linspace(0.0, 1.0, 11):
    plt.vlines(xval, 0.0, 1.1, color='gray', alpha=0.3, linestyles='dashed')
end_time = time.time()
print('\nPlotting and calculating mAP takes {:.4f} secs'.format(end_time - start_time))
plt.show()